In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import os
import sys
import s3fs
import numpy as np
import fireducks.pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import warnings
from dotenv import load_dotenv
from tqdm import tqdm

sys.path.append("../src")
from ml_utils import *

In [9]:
load_dotenv()
pd.set_option("display.max_columns", None)
warnings.simplefilter("ignore")
fs = s3fs.S3FileSystem(
            client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},
            key=os.environ["Accesskey"],
            secret=os.environ["Secretkey"],
            token=os.environ["Token"]
)

In [10]:
with fs.open("elissamim/text_classification_men/data/stages-votes.json", "r") as file:
    df = pd.read_json(file)

df = df.groupby("phrase_text", as_index = False)["sol"].apply(lambda x: x.mode().iloc[0])
df["sol"]=df["sol"].apply(lambda x: 1 if x == "ok" else 0)
df["clean_phrase_text"] = df["phrase_text"].apply(lambda x: nltk_text_preprocessing(x, True))
df = df[df["clean_phrase_text"] != ""]
df.head()

,phrase_text,sol,clean_phrase_text
0,* Aider à la mise en place de l évènement Shar...,0,aider mise place évènemer shareplan envoi rapp...
1,* Comprendre le métier des achats * Comment or...,0,comprendre métier achat comment organiser appe...
2,* Fendre du bois en forêt au merlin manuelleme...,0,fendre boi forêt merlin manuellemer débarder b...
4,"2 jours au CDI , 1 jour en arts plastiques , 1...",0,2 jour cdi 1 jour art plastique 1 jour musiqu ...
5,4 jours au sein du Bureau des affaires institu...,1,4 jour sein bureau affaire institutionnel fina...


In [5]:
X = df["clean_phrase_text"]
y = df["sol"]

static_embedding_models = {
    "Bag of Words":CountVectorizer(),
    "TF-IDF":TfidfVectorizer()
}

classification_models = {
    "Logistic Regression":LogisticRegression(),
    "Random Forest":RandomForestClassifier(),
    "Linear SVM":SVC(kernel="linear", probability=True)
}

dict_scores = {}

for embedding_name, embedding_model in tqdm(static_embedding_models.items(),
                                           desc="Static embeddings"):

    dict_scores[embedding_name] = {}
    
    for classification_name, classification_model in tqdm(classification_models.items(),
                                                         desc="Classification algorithms"):
        
        pipeline = Pipeline(
            [
                ("feature_extraction", embedding_model),
                ("classifier", classification_model)
            ]
        )

        scores = cross_val_score(pipeline, X, y, cv=10, scoring="accuracy")

        dict_scores[embedding_name][classification_name] = f"{np.mean(scores):.3f} ± {np.std(scores):.3f}"

0.6591250854408749